# Лабораторная работа №2. Методы многомерной оптимизации

Цель работы: ознакомление с методами поиска минимума функции двух переменных в оптимизационных задачах без ограничений (метод Гаусса-Зейделя, метод наискорейшего спуска, метод Хука и Дживса, метод Розенброка).

In [60]:
import numpy as np

$$f(x,y)=A-exp\left[-\frac{1}{10-r^2}\left(\frac{(x-a)^2}{c^2}-\frac{2r(x-a)(y-b)}{cd}+\frac{(y-b)^2}{d^2}\right)\right]$$

$$A=20, a=3, b=1, c=3, d=3, r=1$$

In [61]:
def f(x,y):
    return 20 - np.exp(-((x-3)**2 - 2*(x-3)*(y-1) + (y-1)**2) / 81)

Оптимальное значение функции

In [62]:
f(3,1)

19.0

Задача одномерной минимизации

In [63]:
def optimize(x,s,a,b,eps):
  res = [f(*(x+lm*s)) for lm in np.arange(a, b+eps, eps)]
  return a+eps*np.argmin(res)

Метод Гаусса-Зейделя

In [64]:
def gauss_zeidel(x0, eps=0.1):
  x = [np.array(x0)]*2
  s = np.array([[1,0],[0,1]])
  iters = 0
  while True:
    iters += 1
    x[0] = x[1]
    y1 = x[0]
    lm1 = optimize(y1,s[0],-5,5,eps)
    y2 = y1 + lm1 * s[0]
    lm2 = optimize(y2,s[1],-5,5,eps)
    y3 = y2 + lm2 * s[1]
    x[1] = y3

    norm = np.linalg.norm(x[1] - x[0])
    if (norm <= eps): return x[-1].tolist(), f(*x[-1]), iters

Градиент функции

In [65]:
def grad(x,y):
    dfx = (2*x-4-2*y)/(81*np.exp((x**2-4*x+4+y**2+4*y-2*x*y)/81))
    dfy = (2*y+4-2*x)/(81*np.exp((x**2-4*x+4+y**2+4*y-2*x*y)/81))

    return np.array([dfx, dfy])

Значение производной в точке (0,0)

In [66]:
grad(0,0)

array([-0.0470033,  0.0470033])

Значение производной в оптимальной точке (3,1)

In [67]:
grad(3,1)

array([0., 0.])

Метод наискорейшего спуска

In [68]:
def steepest_descent(x1, eps=1e-5):
  x = [np.array(x1)]*2
  iters = 0
  while True:
    iters += 1
    x[0] = x[1]
    gr = grad(*x[0])
    norm = np.linalg.norm(gr)
    if norm < eps: return x[-1].tolist(), f(*x[-1]), iters
    S = -gr/norm
    lmbd = optimize(x[0],S,-5,5,eps)
    x[1] = x[0] + lmbd * S

Метод Хука и Дживса

In [69]:
def hook_n_jeeves(x1, alpha, eps=0.1):
  x = [np.array(x1)]*2
  s = np.array([[1,0],[0,1]])
  iters = 0
  while True:
    iters += 1
    found = False
    x[0] = x[1]
    y1 = x[0]
    if f(*(y1+alpha*s[0])) < f(*y1):
      y2 = y1+alpha*s[0]
      found = True
    elif f(*(y1-alpha*s[0])) < f(*y1):
      y2 = y1-alpha*s[0]
      found = True
    else: y2 = y1

    if f(*(y2+alpha*s[1])) < f(*y2): 
      y3 = y2+alpha*s[1]
      found = True
    elif f(*(y2-alpha*s[1])) < f(*y2): 
      y3 = y2-alpha*s[1]
      found = True
    else: y3 = y2
    
    x[1] = y3
    if alpha < eps: return x[-1].tolist(), f(*x[-1]), iters
    if not found: alpha /= 2

Процедура Грама-Шмидта

In [70]:
def gram_schmidt(lm,s):
  g = [None]*2

  z1 = lm[0]*s[0]+lm[1]*s[1] if lm[0] else s[0]
  z2 = lm[1]*s[1] if lm[1] else s[1]

  g[0] = z1
  g[1] = z2 - np.dot(z2,s[0])*s[0]

  return g/np.linalg.norm(g)

Метод Розенброка

In [71]:
def rozenbrock(x1,eps=0.1):
  x = [np.array(x1)]*2
  s = np.array([[1,0],[0,1]])
  iters = 0
  while True:
    iters += 1
    x[0] = x[1]
    y1 = x[0]
    lm1 = optimize(y1,s[0],-5,5,eps)
    y2 = y1+lm1*s[0]
    lm2 = optimize(y2,s[1],-5,5,eps)
    y3 = y2+lm2*s[1]
    x[1] = y3
    if np.linalg.norm(x[1] - x[0]) < eps:
      return x[-1].tolist(), f(*x[-1]), iters

    y1 = x[1]
    lm = (lm1,lm2)
    s = gram_schmidt(lm,s)

Результаты работы методов многомерной оптимизации:

In [72]:
print("Метод Гаусса-Зейделя:\t", "x* = {}, f(x*) = {} ({} итераций)".format(*gauss_zeidel((6,-7))))
print("Метод наискор.спуска:\t", "x* = {}, f(x*) = {} ({} итераций)".format(*steepest_descent((6,-7))))
print("Метод Хука и Дживса:\t", "x* = {}, f(x*) = {} ({} итераций)".format(*hook_n_jeeves((6,-7), alpha=1)))
print("Метод Розенброка:\t", "x* = {}, f(x*) = {} ({} итераций)".format(*rozenbrock((6,-7))))

Метод Гаусса-Зейделя:	 x* = [0.0, -2.0], f(x*) = 19.0 (3 итераций)
Метод наискор.спуска:	 x* = [0.5000032477782317, -1.5000032477782317], f(x*) = 19.000000000000522 (3 итераций)
Метод Хука и Дживса:	 x* = [0, -2], f(x*) = 19.0 (11 итераций)
Метод Розенброка:	 x* = [0.4997250214691804, -1.4994392571686113], f(x*) = 19.000000008622596 (3 итераций)
